In [1]:
!pip install pandas scikit-learn transformers torch tqdm openpyxl


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load data
df = pd.read_excel('Violation_Training_Data.xlsx')

# Split data
train, val = train_test_split(df, test_size=0.2)

# Tokenize text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train['Combined Text Data'].tolist(), truncation=True, padding=True, return_tensors='pt', max_length=512)
val_encodings = tokenizer(val['Combined Text Data'].tolist(), truncation=True, padding=True, return_tensors='pt', max_length=512)

factor = 'Violation Factor'

# Extract ids, masks, and targets
train_input_ids, train_attention_masks, train_targets = train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train[factor].values)
val_input_ids, val_attention_masks, val_targets = val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val[factor].values)

# Create Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# DataLoaders
batch_size = 16
train_dataloader = DataLoader(TensorDataset(train_input_ids, train_attention_masks, train_targets), batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(TensorDataset(val_input_ids, val_attention_masks, val_targets), batch_size=batch_size)

# Number of training epochs
num_epochs = 30

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs)

# Device & Mixed Precision
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
scaler = torch.cuda.amp.GradScaler()

# Training and Validation Loop
gradient_accumulation_steps = 2  # Adjust for available GPU memory
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_preds, train_labels = [], []

    for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}')):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (batch_idx + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

        total_loss += loss.item() * gradient_accumulation_steps

        # Collect predictions and labels for metrics calculation
        with torch.no_grad():
            preds = torch.argmax(outputs.logits, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(train_dataloader)
    train_acc = accuracy_score(train_labels, train_preds)
    train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_labels, train_preds, average='binary')

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_acc:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')

# Validation step
model.eval()
val_preds, val_labels = [], []
with torch.no_grad():
    for batch in tqdm(val_dataloader, desc='Validating'):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask)  # Corrected line here
        preds = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')
print(f'Validation Accuracy: {val_acc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')

# Save the model and tokenizer
model_save_path = 'bert_model.pt'
tokenizer_save_path = 'bert_tokenizer'

# Save model state_dict
torch.save(model.state_dict(), model_save_path)
# Save tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

print(f'Model saved to {model_save_path}')
print(f'Tokenizer saved to {tokenizer_save_path}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 29/30, Loss: 0.0272, Accuracy: 0.9959, Precision: 0.9909, Recall: 1.0000, F1: 0.9954


Epoch 30/30: 100%|██████████| 16/16 [00:08<00:00,  1.83it/s]


Epoch 30/30, Loss: 0.0179, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000


Validating: 100%|██████████| 4/4 [00:02<00:00,  1.74it/s]


Validation Accuracy: 0.9016, Precision: 0.8571, Recall: 0.9231, F1: 0.8889
Model saved to bert_model.pt
Tokenizer saved to bert_tokenizer
